# Overview
## State Space
- The state space consists of an 80,000-cell grid, representing different geographical locations in Puerto Rico.
- Each cell has attributes like solar PV output, wind power density, elevation, slope, cyclone risk score, building density, road density, and distance to transmission lines.
- Approximately 70% of cells are unavailable for development due to environmental or other constraints.

## Action Space
- Two types of actions are available: building a solar array or a wind turbine.
- Actions can be taken on any available cell.

## Rewards and Costs
The reward function should incorporate:
- Energy production potential (solar and wind).
- Costs or penalties associated with building on certain terrains (e.g., high elevation or steep slopes).
- Penalties for building in high cyclone risk areas.
- Incentives for maintaining a balance between solar and wind energy.
- Incentives for early deployment and distributed grid development.
- Penalties for high building or road density areas.
- Distance to transmission lines.

## RL Model
- Model Choice: Given the size of the state space, a model-based RL algorithm (like Deep Q-Networks or Actor-Critic methods) is suitable.
- Representation: The state representation should include the current status of each grid cell (whether it has a solar array, a wind turbine, or is vacant) along with its attributes.
- Sequence of Actions: The RL agent will sequentially choose actions (where to build next) based on the current state of the grid.
- Terminal State: The agent is done when the environment reaches a certain level of energy capacity or after a fixed number of steps.

# Implementation Steps
## Environment Setup:
- Implement the environment to reflect the grid and its dynamics, including applying the binary mask for unavailable cells.
- The step(action) method should update the grid state based on the chosen action and calculate the immediate reward or cost.

## Agent Development:
- Use PyTorch for implementing the neural network models for the agent.
The agent needs to learn a policy that maximizes long-term rewards, considering the complex reward structure and large state space.

## Training and Evaluation:
- Set up a training loop where the agent interacts with the environment, receives feedback, and improves its policy.
- Periodically evaluate the agent's performance, possibly using separate evaluation episodes or metrics like total energy capacity achieved or adherence to environmental constraints.

## Hyperparameter Tuning:
- Adjust learning rates, exploration rates, discount factors, and network architecture as needed to improve performance.

## Scalability:
Due to the large state space, may need to:
- use function approximation for value functions
- prioritizing important experiences in the replay buffer
- parallelize computation process

## Visualization and Analysis:
- Develop tools to visualize the evolving grid layout and analyze the trade-offs made by the RL agent between different objectives (like energy maximization vs. environmental constraints).

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import torch.optim as optim

from scipy.spatial.distance import cdist, pdist, squareform

import random
import geopandas as gpd
import pandas as pd
import numpy as np

import random
from collections import namedtuple, deque

import copy
import math

In [17]:
# from google.colab import drive
# drive.mount('/content/drive')

# Environment

In [18]:
class RenewableEnergyEnvironment:
    def __init__(self, grid_gdf):
        self.step_count = 0

        # Daily cost calculated in data_preprocessing notebook
        # Multiplying by 4 because grid cell size was changed from 500m to 1 km
        self.wind_daily_cost = 4 * 821.68
        self.solar_daily_cost = 4 * 3007.26
        
        self.grid_columns = ['distance_to_transmission_line', 
                             'cyclone_risk',
                             'occupied']
        self.grid_columns += [f'demand_{i}' for i in range(1,25)]
        self.grid_columns += [f'wind_power_kW_hour_{i}' for i in range(1,25)]
        self.grid_columns += [f'solar_power_kW_hour_{i}' for i in range(1,25)]
        
        self.starting_environment = grid_gdf.copy()
        # Initialize 'occupied' column to zero
        self.starting_environment['occupied'] = 0
        self.starting_environment['installation_type'] = None
        self.state_gdf = self.starting_environment.copy()
        self.state_tensor = self.gdf_to_tensor(self.state_gdf)
        self.mapping, self.action_space_size = self.create_action_to_gdf_mapping()

        self.total_energy_output = 0
        self.stored_energy = 0

        demand_df = pd.read_csv('../data/generation_and_demand/demand_profile.csv')
        self.demand = demand_df['demand_MW'].to_numpy() * 1000
        self.unmet_demand = self.demand.sum()
        self.total_demand = self.unmet_demand

        # LEGACY - the following parameters are no longer in use
        self.decay_rate = 0.1 #TODO determine good decay rate
        self.max_distance = 1000 #TODO get max distance between two cells
        self.weights = {
        'transmission_build_cost': -1.0,
        'early_choice_reward': 1.0,
        'distributed_grid_reward': 1.0,
        }
        
    def reset(self):
        # Reset the environment to the initial state
        self.state_gdf = self.starting_environment.copy()
        self.state_tensor = self.gdf_to_tensor(self.starting_environment)
        self.total_energy_output = 0
        self.step_count = 0
        return self.state_tensor

    def gdf_to_tensor(self, gdf):
        # Calculate grid dimensions
        x_start = 100000
        x_end = 300000
        y_start = 200000
        y_end = 300000
        square_size = 1000
        
        grid_width = int((x_end - x_start) / square_size)
        grid_height = int((y_end - y_start) / square_size)

        transposed_data = self.state_gdf[self.grid_columns].values.T
        grid_data = np.array([feature.reshape(100, 200) for feature in transposed_data])
        tensor = torch.tensor(grid_data, dtype=torch.float16)

        return tensor

    def create_action_to_gdf_mapping(self):
        unmasked_gdf = self.state_gdf[self.state_gdf['masked'] == 0]
        
        mapping = {}
        action_idx = 0  # Initialize action index
    
        for _, row in unmasked_gdf.iterrows():
            # Check for valid solar action
            if row['slope'] <= 8.749:  # Slope check for solar
                mapping[action_idx] = (row.name, 'solar')
                action_idx += 1
    
            # Check for valid wind action
            if row['slope'] <= 26.795:  # Slope check for wind
                mapping[action_idx] = (row.name, 'wind')
                action_idx += 1
        
        action_space_size = action_idx  # Total number of valid actions
        return mapping, action_space_size

    def output_stats(self, writer, cell_index, action_type, reward, invalid=False):
        if writer:
            writer.writerow([None,None,None,cell_index, action_type, int(reward), int(self.total_energy_output.sum()), int(self.unmet_demand)])
        if invalid:
            print(f'Cell: {cell_index:<6d} | Step: INV  | Action: {action_type:<5} | Reward: -9999999 | Energy Output: {int(self.total_energy_output.sum()):<10} | Unmet Demand: {int(self.unmet_demand):<10}')
        else:
            print(f'Cell: {cell_index:<6d} | Step: {self.step_count:<4d} | Action: {action_type:<5} | Reward: {reward:8.5f} | Energy Output: {int(self.total_energy_output.sum()):<10} | Unmet Demand: {int(self.unmet_demand):<10}')

    def step(self, action, writer=None):
        # Apply the action to the environment and return the result
        # action: Tuple (cell_index, action_type) where action_type could be 'solar' or 'wind'

        # Map action from agent output to action in terms of state_gdf
        cell_index, action_type = self.mapping[action]

        # Check if the action is valid
        if not self.is_valid_action(cell_index, action_type):
            reward = -9999999 # Penalty for invalid action
            done = self.is_terminal_state() # Check if in terminal state
            self.total_energy_output = self.calculate_energy_output() # Calculate total energy output
            self.output_stats(writer, cell_index, action_type, reward, invalid=True) # Print results and write results to file
            return self.state_tensor, reward, done, {}

        # Calculate the total reward before applying the action
        total_reward_before_action = self.calculate_reward()
    
        # Apply the action
        self.apply_action(cell_index, action_type)
    
        # Calculate the total reward after applying the action
        total_reward_after_action = self.calculate_reward()
    
        # The reward for the action is the difference in total reward
        reward = total_reward_after_action - total_reward_before_action
        reward = reward / 100000

        # Update total energy output or other state attributes as needed
        self.total_energy_output = self.calculate_energy_output()
        
        # Check if the state is terminal
        done = self.is_terminal_state()

        # Print results and write results to file
        self.output_stats(writer, cell_index, action_type, reward)
        
        # Update the state tensor with the new state gdf
        self.state_tensor = self.gdf_to_tensor(self.state_gdf)

        self.step_count += 1
        
        return self.state_tensor, reward, done, {}

    def is_valid_action(self, cell_index, action_type):
        # Implement logic to check if an action is valid
        cell = self.state_gdf.iloc[cell_index]
        if cell['masked']: # This should never occur
            print('Error: Attempting to build on a masked cell')
            return False
        elif cell['occupied']:
            return False
        elif action_type == 'solar' and cell['slope'] > 8.749: # This should never occur
            print('Error: Attempting to build solar on slope of more than 5%')
            return False
        elif action_type == 'wind' and cell['slope'] > 26.795: # This should never occur
            print('Error: Attempting to build wind on slope of more than 15%')
            return False
        
        return True

    def apply_action(self, cell_index, action_type):
        # Implement the changes to the environment based on the action
        # Example: Mark the cell as occupied and record the type of installation
        self.state_gdf.at[cell_index, 'occupied'] = 1
        self.state_gdf.at[cell_index, 'installation_type'] = action_type

    def calculate_reward(self):
        # Solar installation cost
        solar_cost = self.calculate_solar_cost()

        # Wind turbine installation cost
        wind_cost = self.calculate_wind_cost()
        
        # Solar power Reward
        power_output_reward = self.calculate_power_output_reward()

        # Transmission build cost
        transmission_build_cost = self.calculate_transmission_build_cost()
        
        # Cyclone risk cost
        cyclone_risk_cost = self.calculate_cyclone_risk_cost()
    
        # Distribution reward
        # distributed_grid_reward = self.calculate_distributed_grid_reward()
    
        # Early choice reward
        # early_choice_reward = self.time_dependent_reward_factor()

        total_reward = power_output_reward + solar_cost + wind_cost + cyclone_risk_cost + transmission_build_cost
        
        print(f'Power Output Reward: {power_output_reward:8.5f} | Solar Cost: {solar_cost:8.5f} | Wind Cost: {wind_cost:8.5f} | Transmission Build Cost: {transmission_build_cost:8.5f} | Cyclone Risk Cost: {cyclone_risk_cost:8.5f} | Total Reward: {total_reward:8.5f}')

        return total_reward

    def calculate_energy_output(self):
        # Filter for solar and wind installations
        solar_gdf = self.state_gdf[self.state_gdf['installation_type'] == 'solar']
        wind_gdf = self.state_gdf[self.state_gdf['installation_type'] == 'wind']
    
        # Prepare column names for solar and wind power
        solar_power_columns = [f'solar_power_kW_hour_{i}' for i in range(1, 25)]
        wind_power_columns = [f'wind_power_kW_hour_{i}' for i in range(1, 25)]
    
        # Vectorized sum of power output for solar and wind for each hour
        total_solar_power = solar_gdf[solar_power_columns].sum().to_numpy() * 1000
        total_wind_power = wind_gdf[wind_power_columns].sum().to_numpy() * 1000
    
        total_power = total_solar_power + total_wind_power
        
        return total_power

    def calculate_power_output_reward(self):
        """Uses supply and demand curve to determine the amount of demand satisfied by
        the solar and wind installations"""
        cost_kWh = .22  # TODO get more rigorous number 
        # Storage cost = $400/kWh over the 15 year lifetime of a 4 hour battery
        # cost_storage_kWh = 400 / (365.25 * 15)
        demand = self.demand

        # Calculate the reward using vectorized minimum
        total_power = self.calculate_energy_output()
        demand_satisfied = np.minimum(demand, total_power)
        demand_satisfied_ratio = demand_satisfied / demand

        self.update_demand(demand_satisfied_ratio)
        
        power_reward = demand_satisfied.sum() * cost_kWh
        # self.stored_energy = np.maximum(total_power - demand, 0).sum()
        # storage_cost = self.stored_energy * cost_storage_kWh

        reward = power_reward # - storage_cost
        
        return reward

    def update_demand(self, demand_satisfied_ratio):
        unsatisfied_demand_ratio = 1 - demand_satisfied_ratio
        for i in range(1, 25):
            self.state_gdf[f'demand_{i}'] *= unsatisfied_demand_ratio[i-1]
    
    def calculate_solar_cost(self):
        """Cost of installing solar array on cell. Based on elevation and slope"""
        return -self.solar_daily_cost * len(self.state_gdf[self.state_gdf['installation_type'] == 'solar'])

    def calculate_wind_cost(self):
        """Cost of installing wind turbine on cell. Based on elevation and slope"""
        return -self.wind_daily_cost * len(self.state_gdf[self.state_gdf['installation_type'] == 'wind'])

    def transmission_line_cost_per_km(self, distance):
        # $2.29 million per mile divided by 1.60934 to get into km, 
        # then divided by 25 * 365.25 for daily costs with 25 year decommission time
        distance /= 1000 # Convert to km
        KM_PER_MILE = 1.60934
        COST_PER_KM = 2.29 * 1000000 / (KM_PER_MILE * 25 * 365.25) 
        SHORT_DISTANCE_THRESHOLD = 3 * KM_PER_MILE # Threshold for short distance 
        MEDIUM_DISTANCE_THRESHOLD = 10 * KM_PER_MILE # Threshold for medium distance 
        
        if distance < SHORT_DISTANCE_THRESHOLD:
            cost_modifier = 1.5  # 50% increase for less than 3 miles
        elif distance < MEDIUM_DISTANCE_THRESHOLD:
            cost_modifier = 1.2  # 20% increase for 3-10 miles
        else:
            cost_modifier = 1  # No modification for more than 10 miles
        return -distance * COST_PER_KM * cost_modifier
    
    def calculate_transmission_build_cost(self):
        occupied_cells = self.state_gdf[self.state_gdf['occupied'] == 1]
        
        # Check if there is only one occupied cell
        if len(occupied_cells) == 0:
            return 0
        elif len(occupied_cells) == 1:
            # For a single occupied cell, use the distance to transmission line for cost calculation
            occupied_cell = occupied_cells.iloc[0]
            distance_km = occupied_cell['distance_to_transmission_line'] * 1000
            build_cost = self.transmission_line_cost_per_km(distance_km)
        else:
            # Get coordinates of occupied cells
            coords = np.array(list(zip(occupied_cells.geometry.centroid.x, occupied_cells.geometry.centroid.y)))
    
            # Calculate pairwise distances between occupied cells
            distances = cdist(coords, coords)
    
            # Replace zeros in distance matrix with np.inf to avoid zero distance to itself
            np.fill_diagonal(distances, np.inf)
    
            # Find the nearest installation for each installation
            nearest_installation_distances = np.min(distances, axis=1)
    
            # Determine the relevant distance for cost calculation
            relevant_distances = np.minimum(nearest_installation_distances, occupied_cells['distance_to_transmission_line'].to_numpy()) * 1000
    
            # Calculate build cost
            build_costs = [self.transmission_line_cost_per_km(distance) for distance in relevant_distances]
            build_cost = sum(build_costs)
            
        return build_cost

    def calculate_cyclone_risk_cost(self):
        if len(self.state_gdf[self.state_gdf.installation_type == 'wind']) == 0:
            return 0
        # Wind operational expenses = $40/kW/yr
        # Wind turbine capacity is 3 MW
        # To get daily cost, multiply by 3000 (3 MW = 3000 kW) and divide by 365.25 days in a year
        wind_opex = 4 * 40 * 3000 / 365.25
        # Wind capital expenses = $1501/kW
        # To get daily cost, multiply by 3000 (3 MW = 3000 kW) and divide by (25 years of lifetime * 365.25 days in a year)
        wind_capex = 4 * 1501 * 3000 / (25 * 365.25)
        cyclone_risk_cost = -(self.state_gdf[self.state_gdf.installation_type == 'wind']['cyclone_risk'] * (wind_capex - wind_opex)).sum()
        return cyclone_risk_cost
    
    def calculate_distributed_grid_reward(self):
        # Extract the coordinates of the occupied cells (where installations are located)
        occupied_cells = self.state_gdf[self.state_gdf['occupied'] == 1]
        if len(occupied_cells) < 2:
            # If there are less than two installations, we cannot calculate distances
            return 1
    
        coords = np.array(list(zip(occupied_cells.geometry.x, occupied_cells.geometry.y)))
    
        # Calculate pairwise distances between all occupied cells
        distances = pdist(coords)
    
        # Calculate the average distance. The larger this is, the more distributed the installations are.
        avg_distance = np.mean(distances)
    
        # Normalize the reward such that it ranges between 0 and 1
        normalized_reward = avg_distance / self.max_distance
    
        return normalized_reward
    
    def time_dependent_reward_factor(self):
        # A function to calculate the time-dependent reward factor
        # It decreases with each year from the base year
        action_number = self.state_gdf.occupied.sum()
        return 1 / (1 + self.decay_rate * action_number)
        
    def is_terminal_state(self):
        # The episode ends when there is no unmet demand
        self.unmet_demand = np.maximum(self.demand - self.total_energy_output, 0).sum()
        return self.unmet_demand <= 0 or self.total_energy_output.sum() >= self.total_demand 

    def render(self):
        # Optional: Implement a method to visualize the current state of the environment
        pass

In [19]:
# class RenewableEnergyEnvironment:
#     def __init__(self, grid_gdf):
#         self.weights = {
#         'transmission_build_cost': -1.0,
#         'early_choice_reward': 1.0,
#         'distributed_grid_reward': 1.0,
#         }

#         self.wind_daily_cost = 821.68
#         self.solar_daily_cost = 3007.26

#         self.grid_columns = ['slope',
#                              'distance_to_transmission_line',
#                              'cyclone_risk',
#                              'water',
#                              'occupied_solar',
#                              'occupied_wind',
#                              'masked']
#         self.grid_columns += [f'demand_{i}' for i in range(1,25)]
#         self.grid_columns += [f'wind_power_kW_hour_{i}' for i in range(1,25)]
#         self.grid_columns += [f'solar_power_kW_hour_{i}' for i in range(1,25)]

#         # Initialize the environment
#         self.starting_environment = grid_gdf
#         self.state_gdf = grid_gdf.copy()
#         self.state_tensor = self.gdf_to_tensor(self.state_gdf)
#         self.mapping, self.action_space_size = self.create_action_to_gdf_mapping()

#         self.total_energy_output = 0
#         self.stored_energy = 0

#         demand_df = pd.read_csv('../data/generation_and_demand/demand_profile.csv')
#         # demand_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/GreenGridPR/data/generation_and_demand/demand_profile.csv')
#         self.demand = demand_df['demand_MW'].to_numpy() * 1000
#         self.unmet_demand = self.demand.sum()
#         self.total_demand = self.unmet_demand

#         self.decay_rate = 0.1 #TODO determine good decay rate
#         self.max_distance = 1000 #TODO get max distance between two cells

#     def reset(self):
#         # Reset the environment to the initial state
#         self.state_gdf = self.starting_environment.copy()
#         self.state_tensor = self.gdf_to_tensor(self.starting_environment)
#         self.total_energy_output = 0
#         return self.state_tensor

#     def gdf_to_tensor(self, gdf):
#         # Calculate grid dimensions
#         x_start = 100000
#         x_end = 300000
#         y_start = 200000
#         y_end = 300000
#         square_size = 500

#         grid_width = int((x_end - x_start) / square_size)
#         grid_height = int((y_end - y_start) / square_size)

#         # Flatten the grid data
#         flat_data = self.state_gdf[self.grid_columns].values.reshape(-1, grid_height, grid_width)

#         # Create a 4D Tensor (batch size is the 4th dimension)
#         tensor = torch.tensor(flat_data, dtype=torch.float32)

#         return tensor

#     def create_action_to_gdf_mapping(self):
#         unmasked_gdf = self.state_gdf[self.state_gdf['masked'] == 0]

#         mapping = {}
#         action_idx = 0  # Initialize action index

#         for _, row in unmasked_gdf.iterrows():
#             # Check for valid solar action
#             if row['slope'] <= 8.749:  # Slope check for solar
#                 mapping[action_idx] = (row.name, 'solar')
#                 action_idx += 1

#             # Check for valid wind action
#             if row['slope'] <= 26.795:  # Slope check for wind
#                 mapping[action_idx] = (row.name, 'wind')
#                 action_idx += 1

#         action_space_size = action_idx  # Total number of valid actions
#         return mapping, action_space_size


#     def step(self, action):
#         # Apply the action to the environment and return the result
#         # action: Tuple (cell_index, action_type) where action_type could be 'solar' or 'wind'

#         cell_index, action_type = self.mapping[action]

#         # Check if the action is valid
#         if not self.is_valid_action(cell_index, action_type):
#             reward = -1  # Penalty for invalid action
#             done = self.is_terminal_state()
#             return self.state_tensor, reward, done, {}

#         # Calculate the total reward before applying the action
#         total_reward_before_action = self.calculate_reward()

#         # Apply the action
#         self.apply_action(cell_index, action_type)

#         # Calculate the total reward after applying the action
#         total_reward_after_action = self.calculate_reward()

#         # The reward for the action is the difference in total reward
#         reward = total_reward_after_action - total_reward_before_action

#         # Update total energy output or other state attributes as needed
#         self.total_energy_output = self.calculate_energy_output()

#         # Check if the state is terminal
#         done = self.is_terminal_state()

#         print(f'Cell: {cell_index:<6d} | Action: {action_type:<5} | Reward: {int(reward):<7} | Energy Output: {int(self.total_energy_output.sum()):<10} | Unmet Demand: {int(self.unmet_demand):<10}')

#         # Update the state tensor with the new state gdf
#         self.state_tensor = self.gdf_to_tensor(self.state_gdf)

#         return self.state_tensor, reward, done, {}

#     def is_valid_action(self, cell_index, action_type):
#         # Implement logic to check if an action is valid
#         # Example: Check if the cell is not masked and not already occupied
#         cell = self.state_gdf.iloc[cell_index]
#         if cell['masked']: # This should never occur
#             print('Something has gone wrong. Attempting to build on a masked cell')
#             return False
#         elif cell['occupied_solar']:
#             return False
#         elif cell['occupied_wind']:
#             return False
#         elif action_type == 'solar' and cell['slope'] > 8.749:
#             return False
#         elif action_type == 'wind' and cell['slope'] > 26.795:
#             return False

#         return True

#     def apply_action(self, cell_index, action_type):
#         # Implement the changes to the environment based on the action
#         # Example: Mark the cell as occupied and record the type of installation
#         if action_type == 'solar':
#             self.state_gdf.at[cell_index, 'occupied_solar'] = 1
#         elif action_type == 'wind':
#             self.state_gdf.at[cell_index, 'occupied_wind'] = 1

#     def calculate_reward(self):
#         # Solar installation cost
#         solar_cost = self.calculate_solar_cost()

#         # Wind turbine installation cost
#         wind_cost = self.calculate_wind_cost()

#         # Solar power Reward
#         power_output_reward = self.calculate_power_output_reward()

#         # Transmission build cost
#         transmission_build_cost = self.calculate_transmission_build_cost()

#         # Cyclone risk cost
#         cyclone_risk_cost = self.calculate_cyclone_risk_cost()

#         # Distribution reward
#         # distributed_grid_reward = self.calculate_distributed_grid_reward()

#         # Early choice reward
#         # early_choice_reward = self.time_dependent_reward_factor()

#         total_reward = power_output_reward + solar_cost + wind_cost + cyclone_risk_cost + transmission_build_cost

#         return total_reward

#     def calculate_energy_output(self):
#         # Filter for solar and wind installations
#         solar_gdf = self.state_gdf[self.state_gdf['occupied_solar'] == 1]
#         wind_gdf = self.state_gdf[self.state_gdf['occupied_wind'] == 1]

#         # Prepare column names for solar and wind power
#         solar_power_columns = [f'solar_power_kW_hour_{i}' for i in range(1, 25)]
#         wind_power_columns = [f'wind_power_kW_hour_{i}' for i in range(1, 25)]

#         # Vectorized sum of power output for solar and wind for each hour
#         total_solar_power = solar_gdf[solar_power_columns].sum().to_numpy()
#         total_wind_power = wind_gdf[wind_power_columns].sum().to_numpy()

#         total_power = total_solar_power + total_wind_power

#         return total_power

#     def calculate_power_output_reward(self):
#         """Uses supply and demand curve to determine the amount of demand satisfied by
#         the solar and wind installations"""
#         cost_kWh = .22  # TODO
#         # Storage cost = $400/kWh over the 15 year lifetime of a 4 hour battery
#         # cost_storage_kWh = 400 / (365.25 * 15)
#         demand = self.demand

#         # Calculate the reward using vectorized minimum
#         total_power = self.calculate_energy_output()
#         demand_satisfied = np.minimum(demand, total_power)
#         demand_satisfied_ratio = demand_satisfied / demand

#         self.update_demand(demand_satisfied_ratio)

#         power_reward = demand_satisfied.sum() * cost_kWh
#         # self.stored_energy = np.maximum(total_power - demand, 0).sum()
#         # storage_cost = self.stored_energy * cost_storage_kWh

#         reward = power_reward # - storage_cost

#         return reward

#     def update_demand(self, demand_satisfied_ratio):
#         unsatisfied_demand_ratio = 1 - demand_satisfied_ratio
#         for i in range(1, 25):
#             self.state_gdf[f'demand_{i}'] *= unsatisfied_demand_ratio[i-1]

#     def calculate_solar_cost(self):
#         """Cost of installing solar array on cell. Based on elevation and slope"""
#         return -self.solar_daily_cost * len(self.state_gdf[self.state_gdf['occupied_solar'] == 1])

#     def calculate_wind_cost(self):
#         """Cost of installing wind turbine on cell. Based on elevation and slope"""
#         return -self.wind_daily_cost * len(self.state_gdf[self.state_gdf['occupied_wind'] == 1])

#     def transmission_line_cost_per_km(self, distance):
#         # $2.29 million per mile divided by 1.60934 to get into km,
#         # then divided by 25 * 365.25 for daily costs with 25 year decommission time
#         distance /= 1000 # Convert to km
#         KM_PER_MILE = 1.60934
#         COST_PER_KM = 2.29 * 1000000 / (KM_PER_MILE * 25 * 365.25)
#         SHORT_DISTANCE_THRESHOLD = 3 * KM_PER_MILE # Threshold for short distance
#         MEDIUM_DISTANCE_THRESHOLD = 10 * KM_PER_MILE # Threshold for medium distance

#         if distance < SHORT_DISTANCE_THRESHOLD:
#             cost_modifier = 1.5  # 50% increase for less than 3 miles
#         elif distance < MEDIUM_DISTANCE_THRESHOLD:
#             cost_modifier = 1.2  # 20% increase for 3-10 miles
#         else:
#             cost_modifier = 1  # No modification for more than 10 miles
#         return -distance * COST_PER_KM * cost_modifier

#     def calculate_transmission_build_cost(self):
#         occupied_cells = self.state_gdf[(self.state_gdf['occupied_solar'] == 1) | (self.state_gdf['occupied_wind'] == 1)]

#         # Check if there is only one occupied cell
#         if len(occupied_cells) == 0:
#             return 0
#         elif len(occupied_cells) == 1:
#             # For a single occupied cell, use the distance to transmission line for cost calculation
#             occupied_cell = occupied_cells.iloc[0]
#             distance_km = occupied_cell['distance_to_transmission_line']
#             build_cost = self.transmission_line_cost_per_km(distance_km)
#         else:
#             # Get coordinates of occupied cells
#             coords = np.array(list(zip(occupied_cells.geometry.centroid.x, occupied_cells.geometry.centroid.y)))

#             # Calculate pairwise distances between occupied cells
#             distances = cdist(coords, coords)

#             # Replace zeros in distance matrix with np.inf to avoid zero distance to itself
#             np.fill_diagonal(distances, np.inf)

#             # Find the nearest installation for each installation
#             nearest_installation_distances = np.min(distances, axis=1)

#             # Determine the relevant distance for cost calculation
#             relevant_distances = np.minimum(nearest_installation_distances, occupied_cells['distance_to_transmission_line'].to_numpy())

#             # Calculate build cost
#             build_costs = [self.transmission_line_cost_per_km(distance) for distance in relevant_distances]
#             build_cost = sum(build_costs)

#         return build_cost

#     def calculate_cyclone_risk_cost(self):
#         if len(self.state_gdf[self.state_gdf.occupied_wind == 1]) == 0:
#             return 0
#         # Wind operational expenses = $40/kW/yr
#         # Wind turbine capacity is 3 MW
#         # To get daily cost, multiply by 3000 (3 MW = 3000 kW) and divide by 365.25 days in a year
#         wind_opex = 40 * 3000 / 365.25
#         # Wind capital expenses = $1501/kW
#         # To get daily cost, multiply by 3000 (3 MW = 3000 kW) and divide by (25 years of lifetime * 365.25 days in a year)
#         wind_capex = 1501 * 3000 / (25 * 365.25)
#         cyclone_risk_cost = -(self.state_gdf[self.state_gdf.occupied_wind == 1]['cyclone_risk'] * (wind_capex - wind_opex)).sum()
#         return cyclone_risk_cost

#     def calculate_distributed_grid_reward(self):
#         # Extract the coordinates of the occupied cells (where installations are located)
#         occupied_cells = self.state_gdf[(self.state_gdf['occupied_solar'] == 1) | (self.state_gdf['occupied_wind'] == 1)]
#         if len(occupied_cells) < 2:
#             # If there are less than two installations, we cannot calculate distances
#             return 1

#         coords = np.array(list(zip(occupied_cells.geometry.x, occupied_cells.geometry.y)))

#         # Calculate pairwise distances between all occupied cells
#         distances = pdist(coords)

#         # Calculate the average distance. The larger this is, the more distributed the installations are.
#         avg_distance = np.mean(distances)

#         # Normalize the reward such that it ranges between 0 and 1
#         normalized_reward = avg_distance / self.max_distance

#         return normalized_reward

#     def time_dependent_reward_factor(self):
#         # A function to calculate the time-dependent reward factor
#         # It decreases with each year from the base year
#         action_number = self.state_gdf.occupied_solar.sum() + self.state_gdf.occupied_wind.sum()
#         return 1 / (1 + self.decay_rate * action_number)

#     def is_terminal_state(self):
#         # The episode ends when there is no unmet demand
#         self.unmet_demand = np.maximum(self.demand - self.total_energy_output, 0).sum()
#         # self.unmet_demand -= self.stored_energy
#         return self.unmet_demand <= 0 or self.total_energy_output.sum() >= 2 * self.total_demand

#     def render(self):
#         # Optional: Implement a method to visualize the current state of the environment
#         pass

NN Architecture

In [20]:
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, net_width):
        super(Actor, self).__init__()

        # self.conv1 = nn.Conv2d(state_dim, 128, kernel_size=3, stride=2)
        # self.bn1 = nn.BatchNorm2d(128)
        # self.conv2 = nn.Conv2d(128, 128, kernel_size=3, stride=2)
        # self.bn2 = nn.BatchNorm2d(128)
        # self.conv3 = nn.Conv2d(128, 128, kernel_size=3, stride=2)
        # self.bn3 = nn.BatchNorm2d(128)
        
        self.num_nontemporal_channels = 2
        self.num_temporal_channels = state_dim[0] - self.num_nontemporal_channels  # Subtracting the non-temporal channels
        print(state_dim[0])
        self.time_dimension = 24
        self.grid_height = state_dim[1]
        self.grid_width = state_dim[2]

        # 3D Convolutional layers for temporal features
        # print(self.num_temporal_channels // 24)
        self.conv3d1 = nn.Conv3d(self.num_temporal_channels // 24, 3, kernel_size=(3, 3, 3), stride=(1, 2, 2))
        self.bn3d1 = nn.BatchNorm3d(3)
        self.conv3d2 = nn.Conv3d(3, 3, kernel_size=(3, 3, 3), stride=(1, 2, 2))
        self.bn3d2 = nn.BatchNorm3d(3)
        # Add more layers if necessary

        # 2D Convolutional layers for non-temporal features
        self.conv2d1 = nn.Conv2d(self.num_nontemporal_channels, self.num_nontemporal_channels, kernel_size=3, stride=2)
        self.bn2d1 = nn.BatchNorm2d(self.num_nontemporal_channels)
        self.conv2d2 = nn.Conv2d(self.num_nontemporal_channels, self.num_nontemporal_channels, kernel_size=3, stride=2)
        self.bn2d2 = nn.BatchNorm2d(self.num_nontemporal_channels)
        # Add more layers if necessary

        # Fully connected layer for processing 'occupied' cells feature
        self.fc_occupied = nn.Linear(self.grid_height * self.grid_width, 1024)

        # Calculate the size of the combined feature vector
        combined_feature_size = self.calculate_combined_feature_size(state_dim)

        # Fully connected layers for decision making
        self.fc1 = nn.Linear(73936, 1024)
        self.fc2 = nn.Linear(1024, action_dim)

    def calculate_combined_feature_size(self, state_dim):
        # Dummy inputs for calculating feature sizes
        dummy_temporal = torch.zeros(1, 72 // 24, self.time_dimension, self.grid_height, self.grid_width)
        dummy_nontemporal = torch.zeros(1, self.num_nontemporal_channels, self.grid_height, self.grid_width)
        dummy_occupied = torch.zeros(1, 1, self.grid_height, self.grid_width)

        # Forward pass through convolutional layers
        x_temporal = self.bn3d2(self.conv3d2(self.bn3d1(self.conv3d1(dummy_temporal))))
        x_nontemporal = self.bn2d2(self.conv2d2(self.bn2d1(self.conv2d1(dummy_nontemporal))))
        x_occupied = dummy_occupied.view(1, -1)

        # Calculate the total number of features
        total_features = x_temporal.numel() + x_nontemporal.numel() + x_occupied.numel()
        return total_features
    
    #     self.input_shape = state_dim  # Store input_shape for feature size calculation
    #     print(f'Input shape: {state_dim}, n Actions: {action_dim}')
    #     self.fc1 = nn.Linear(33792, 1024)
    #     self.fc2 = nn.Linear(1024, action_dim)

    def pi(self, state, softmax_dim = 0):
        x =  state.view(-1, 75, 100, 200)
        print( f'x1: {x}')

        # Split input into temporal and non-temporal components
        x_temporal = x[:, 3:, :, :].view(x.size(0), self.num_temporal_channels // 24, self.time_dimension, self.grid_height, self.grid_width)
        print( f'x_temporal1: {x_temporal}')

        x_nontemporal = x[:, :2, :, :]  # Distance to transmission line & cyclone risk
        print( f'x_nontemporal1: {x_nontemporal}')

        x_occupied = x[:, 2:3, :, :].view(x.size(0), -1)  # Occupied cells
        print( f'x_occupied: {x_occupied}')

        # Process temporal features
        x_temporal = F.relu(self.bn3d1(self.conv3d1(x_temporal)))
        print( f'x_temporal2: {x_temporal}')

        x_temporal = F.relu(self.bn3d2(self.conv3d2(x_temporal)))
        print( f'x_temporal2: {x_temporal}')

        x_temporal = x_temporal.view(x_temporal.size(0), -1)
        print( f'x_temporal2: {x_temporal}')

        # Process non-temporal features
        x_nontemporal = F.relu(self.bn2d1(self.conv2d1(x_nontemporal)))
        print( f'x_nontemporal2: {x_nontemporal}')

        x_nontemporal = F.relu(self.bn2d2(self.conv2d2(x_nontemporal)))
        print( f'x_nontemporal3: {x_nontemporal}')

        x_nontemporal = x_nontemporal.view(x_nontemporal.size(0), -1)
        print( f'x_nontemporal4: {x_nontemporal}')

        # Process 'occupied' cells feature
        x_occupied = F.relu(self.fc_occupied(x_occupied))
        print(f'x_occupied2: {x_occupied}')

        # Combine features
        x_combined = torch.cat((x_temporal, x_nontemporal, x_occupied), dim=1)
        print(f'x_combined1: {x_combined}')

        # Fully connected layers
        x_combined = F.relu(self.fc1(x_combined))
        print(f'x_combined2: {x_combined}')

        return self.fc2(x_combined)
    
    #     print('start : ',state.size())
    #     print('start : ',state)

        # x =  state.view(-1, 75, 100, 200)
        
    #     print( f'x1 shape: {x.size()}')
    #     print( f'x1 shape: {x}')

    #     x = F.relu(self.bn1(self.conv1(x)))
    #     print( f'x2 shape: {x.size()}')
    #     print( f'x2 shape: {x}')

    #     x = F.relu(self.bn2(self.conv2(x)))
    #     print( f'x3 shape: {x.size()}')
    #     print( f'x3 shape: {x}')

    #     x = F.relu(self.bn3(self.conv3(x)))
    #     print( f'x4 shape: {x.size()}')
    #     print( f'x4 shape: {x}')

    #     x = x.view(x.size(0), -1)
    #     print( f'x5 shape: {x.size()}')
    #     print( f'x5 shape: {x}')

    #     x = F.relu(self.fc1(x))
    #     print( f'x6 shape: {x.size()}')
    #     print( f'x6 shape: {x}')

    #     x = self.fc2(x)
    #     print( f'x shape: {x.size()}')
    #     print( f'x shape: {x}')
        
    #     prob = F.softmax(x,dim=softmax_dim)
    #     print( f'prob shape: {prob.size()}')
    #     print( f'prob: {prob}')
    #     return prob

class Critic(nn.Module):
    def __init__(self, state_dim,net_width):
        super(Critic, self).__init__()

        self.C1 = nn.Linear(state_dim, net_width)
        self.C2 = nn.Linear(net_width, net_width)
        self.C3 = nn.Linear(net_width, 1)

    def forward(self, state):
        v = torch.relu(self.C1(state))
        v = torch.relu(self.C2(v))
        v = self.C3(v)
        return v

def evaluate_policy(env, agent, turns = 3):
    total_scores = 0
    for j in range(turns):
        s, info = env.reset()
        done = False
        while not done:
            # Take deterministic actions at test time
            a, logprob_a = agent.select_action(s, deterministic=True)
            s_next, r, dw, tr, info = env.step(a)
            done = (dw or tr)

            total_scores += r
            s = s_next
    return int(total_scores/turns)

In [21]:
class PPO_discrete():
    def __init__(self, **kwargs):
        required_params = ['state_dim', 'action_dim', 'net_width', 'lr', 'device']
        for param in required_params:
            if param not in kwargs:
                raise ValueError(f"Parameter '{param}' must be specified")

        # Set attributes from kwargs
        self.state_dim = kwargs['state_dim']
        self.action_dim = kwargs['action_dim']
        self.net_width = kwargs['net_width']
        self.lr = kwargs['lr']
        self.dvc = kwargs['device']
        self.T_horizon = kwargs['T_horizon']

        # print('state dim :  ',self.state_dim.size())
        # print('action dim :  ',self.action_dim)
        # print('net width :  ',self.net_width)

        '''Build Actor and Critic'''
        self.actor = Actor(self.state_dim, self.action_dim, self.net_width).to(self.dvc)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=self.lr)
        self.critic = Critic(self.state_dim[0], self.net_width).to(self.dvc)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=self.lr)

        '''Build Trajectory holder'''
        self.s_hoder = np.zeros((self.T_horizon, 75, 100, 200), dtype=np.float16)
        self.a_hoder = np.zeros((self.T_horizon, 1), dtype=np.int64)
        self.r_hoder = np.zeros((self.T_horizon, 1), dtype=np.float16)
        self.s_next_hoder = np.zeros((self.T_horizon, 75, 100, 200), dtype=np.float16)
        self.logprob_a_hoder = np.zeros((self.T_horizon, 1), dtype=np.float16)
        self.done_hoder = np.zeros((self.T_horizon, 1), dtype=np.bool_)
        self.dw_hoder = np.zeros((self.T_horizon, 1), dtype=np.bool_)

    def select_action(self, s, deterministic):
        s = s.float().to(self.dvc)
        with torch.no_grad():
            pi = self.actor.pi(s, softmax_dim=1)
            
            if deterministic:
                a = torch.argmax(pi, dim=1).item()
                return a, None
            else:
                print(pi)
                m = Categorical(pi)
                a = m.sample().item()
            

                pi_a = pi[0,a].item()
                return a, pi_a

    def train(self):
        self.entropy_coef *= self.entropy_coef_decay #exploring decay
        '''Prepare PyTorch data from Numpy data'''
        s = torch.from_numpy(self.s_hoder).to(self.dvc)
        a = torch.from_numpy(self.a_hoder).to(self.dvc)
        r = torch.from_numpy(self.r_hoder).to(self.dvc)
        s_next = torch.from_numpy(self.s_next_hoder).to(self.dvc)
        old_prob_a = torch.from_numpy(self.logprob_a_hoder).to(self.dvc)
        done = torch.from_numpy(self.done_hoder).to(self.dvc)
        dw = torch.from_numpy(self.dw_hoder).to(self.dvc)

        ''' Use TD+GAE+LongTrajectory to compute Advantage and TD target'''
        with torch.no_grad():
            vs = self.critic(s)
            vs_ = self.critic(s_next)

            '''dw(dead and win) for TD_target and Adv'''
            deltas = r + self.gamma * vs_ * (~dw) - vs
            deltas = deltas.cpu().flatten().numpy()
            adv = [0]

            '''done for GAE'''
            for dlt, done in zip(deltas[::-1], done.cpu().flatten().numpy()[::-1]):
                advantage = dlt + self.gamma * self.lambd * adv[-1] * (~done)
                adv.append(advantage)
            adv.reverse()
            adv = copy.deepcopy(adv[0:-1])
            adv = torch.tensor(adv).unsqueeze(1).float().to(self.dvc)
            td_target = adv + vs
            if self.adv_normalization:
                adv = (adv - adv.mean()) / ((adv.std() + 1e-4))  #sometimes helps

        """PPO update"""
        #Slice long trajectopy into short trajectory and perform mini-batch PPO update
        optim_iter_num = int(math.ceil(s.shape[0] / self.batch_size))

        for _ in range(self.K_epochs):
            #Shuffle the trajectory, Good for training
            perm = np.arange(s.shape[0])
            np.random.shuffle(perm)
            perm = torch.LongTensor(perm).to(self.dvc)
            s, a, td_target, adv, old_prob_a = \
                s[perm].clone(), a[perm].clone(), td_target[perm].clone(), adv[perm].clone(), old_prob_a[perm].clone()

            '''mini-batch PPO update'''
            for i in range(optim_iter_num):
                index = slice(i * self.batch_size, min((i + 1) * self.batch_size, s.shape[0]))

                '''actor update'''
                prob = self.actor.pi(s[index], softmax_dim=1)
                entropy = Categorical(prob).entropy().sum(0, keepdim=True)
                prob_a = prob.gather(1, a[index])
                ratio = torch.exp(torch.log(prob_a) - torch.log(old_prob_a[index]))  # a/b == exp(log(a)-log(b))

                surr1 = ratio * adv[index]
                surr2 = torch.clamp(ratio, 1 - self.clip_rate, 1 + self.clip_rate) * adv[index]
                a_loss = -torch.min(surr1, surr2) - self.entropy_coef * entropy

                self.actor_optimizer.zero_grad()
                a_loss.mean().backward()
                torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 40)
                self.actor_optimizer.step()

                '''critic update'''
                c_loss = (self.critic(s[index]) - td_target[index]).pow(2).mean()
                for name, param in self.critic.named_parameters():
                    if 'weight' in name:
                        c_loss += param.pow(2).sum() * self.l2_reg

                self.critic_optimizer.zero_grad()
                c_loss.backward()
                self.critic_optimizer.step()

    def put_data(self, s, a, r, s_next, logprob_a, done, dw, idx):
        # print(logprob_a)
        # print(done)
        # print(dw)
        # print(idx)

        self.s_hoder[idx] = s
        self.a_hoder[idx] = a
        self.r_hoder[idx] = r
        self.s_next_hoder[idx] = s_next
        self.logprob_a_hoder[idx] = logprob_a
        self.done_hoder[idx] = done
        self.dw_hoder[idx] = dw

    def save(self, episode):
        torch.save(self.critic.state_dict(), "./model/ppo_critic{}.pth".format(episode))
        torch.save(self.actor.state_dict(), "./model/ppo_actor{}.pth".format(episode))

    def load(self, episode):
        self.critic.load_state_dict(torch.load("./model/ppo_critic{}.pth".format(episode)))
        self.actor.load_state_dict(torch.load("./model/ppo_actor{}.pth".format(episode)))


In [22]:
# state_gdf = gpd.read_parquet('/content/drive/MyDrive/Colab Notebooks/GreenGridPR/data/processed/state.parquet')
state_gdf = gpd.read_parquet('../data/processed/state_1km_grid_cells.parquet')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

environment = RenewableEnergyEnvironment(state_gdf)
state_space = environment.state_tensor.to(device)
action_space_size = environment.action_space_size

episodes = 16
learning_rate = 0.000001 

T_horizon_value = 2048 # number of interactions agent collects before updating its policy network

# Example of initializing PPO agent
ppo_agent = PPO_discrete(state_dim=environment.state_tensor.size(), 
                         action_dim=environment.action_space_size, 
                         net_width=128,
                         device=device, 
                         lr = learning_rate,
                         T_horizon=T_horizon_value)
idx = 0
for episode in range(episodes):
    state = environment.reset()
    done = False

    while not done:
        action, logprob_action = ppo_agent.select_action(state, deterministic=False)
        next_state, reward, done, info = environment.step(action)

        # Store data in PPO buffer
        ppo_agent.put_data(state, action, reward, next_state, logprob_action, done, done, idx)
        idx += 1

        state = next_state
    
    # After each episode, train the PPO agent
    ppo_agent.train()

Using device: cuda
75
x1: tensor([[[[3.9200e+04, 3.8432e+04, 3.7632e+04,  ..., 3.8848e+04,
           3.9648e+04, 4.0512e+04],
          [3.8016e+04, 3.7216e+04, 3.6416e+04,  ..., 3.7728e+04,
           3.8592e+04, 3.9456e+04],
          [3.6896e+04, 3.6064e+04, 3.5232e+04,  ..., 3.6704e+04,
           3.7600e+04, 3.8464e+04],
          ...,
          [3.8272e+04, 3.7728e+04, 3.7216e+04,  ..., 4.0864e+04,
           4.1792e+04, 4.2720e+04],
          [3.9968e+04, 3.9456e+04, 3.8944e+04,  ..., 4.1632e+04,
           4.2560e+04, 4.3456e+04],
          [4.1664e+04, 4.1184e+04, 4.0704e+04,  ..., 4.2496e+04,
           4.3392e+04, 4.4288e+04]],

         [[2.9370e-01, 2.9370e-01, 2.9370e-01,  ..., 1.0681e-01,
           1.0681e-01, 1.0681e-01],
          [2.6709e-01, 2.6709e-01, 2.6709e-01,  ..., 1.0681e-01,
           1.0681e-01, 1.0681e-01],
          [2.6709e-01, 2.6709e-01, 2.6709e-01,  ..., 1.0681e-01,
           1.0681e-01, 1.0681e-01],
          ...,
          [2.6709e-01, 2.6709e-01

ValueError: Expected parameter probs (Tensor of shape (1, 4849)) of distribution Categorical(probs: torch.Size([1, 4849])) to satisfy the constraint Simplex(), but found invalid values:
tensor([[nan, nan, nan,  ..., nan, nan, nan]], device='cuda:0')